<center><h1>Tree-Based Models</h1></center>
<center><h3>2023-03-15</h3></center>

# What are "Tree-based Methods"?

  * Species of supervised machine learning models
  * These models are more "algorithmic", not so formulaic
  * Basic idea is to build "decision trees"

## Decision Trees

<div align="center">
<img src="images/amiahorse.jpg" width=480/>
</div>


## Species of Tree-Based Methods

  * ID3 (Quinlan, 1986)
  * C4.5 (Quinlan, 1993)
  * C5.0 (Quinlan, 1996)
  * CART (Breiman, Friedman, Olshen, & Stone, 1984)

# Classification and Regression Trees

* A decision tree can be used for both classification and regression tasks
* Tree's internal node represents a test on an attribute or feature, each branch represents the outcome of the test, and each leaf node represents a class label or a target value.
* Tree-building algorithms recursively partition the feature space into smaller subspaces based on the values of the features, using the most informative feature at each step to create the split
  - The process continues until a stopping criterion is met, such as reaching a certain depth or having a minimum number of samples in each leaf node.
* Advantages of decision trees:
  - Easy to understand and interpret
  - Can handle both categorical and continuous input features
  - Can capture complex relationships between the features and the target variable

### CART Algorithm Pseudo-code
![image](images/cart_pseudo_code.png)

## Titanic Example

  * Titantic survival data
  * Kaggle competition
  * Predicting survival (2-class problem)
  * Using demographic and other variables

In [ ]:
import pandas as pd

titanic_df = pd.read_csv("data/titanic_subset.csv")

titanic_df.head()

### Titanic CART Tree
<img src=images/tree.png height="500">

### Advantages of CART Trees
  * Intuitive
  * Viable when $n \ll p$
  * Handle interactions naturally
  * Minimal assumptions
  * Handle missingness in predictors

### Disadvantages of Single Trees
  * High variance
  * Prone to overfitting
  * Lack of smoothness (troubling for regression)

## Bootstrap (Quick Detour)
* The bootstrap is a resampling technique used to estimate the sampling distribution of a statistic or to quantify the uncertainty associated with a point estimate
* Involves drawing a large number of random samples with replacement from the original dataset to create "bootstrap samples"
* For each bootstrap sample, the statistic of interest (such as the mean or median) is computed, and the distribution of these statistics across all bootstrap samples is used to estimate the sampling distribution of the statistic.
* Can be used to compute confidence intervals and p-values without making distributional assumptions 

# Bootstrap Aggregation ("Bagging")
  * Breiman (1996)
  * Extends idea of CART models
    - Single trees overfit
    - Stopping rules and pruning help, but only to a point
  * Take $B$ bootstrap samples, build $B$ trees, aggregate predictions
    - For regression, aggregation is taking the mean of the predicted values for each $y_i$
	- For classification, to aggregate means each tree casts a ``vote'' for each $y_i$

## Bagging Pseudo-Code
![image](images/bagging_pseudo_code.png)

## Advantages of Bagging
  * Big improvement in predictive performance
  * Variance decreases
  * Easy to tune
  * Embarrassingly parallel
  * Out-of-bag (OOB) error estimate (more on this later)

<center><h1>Challenge Problem</h1><center>
    
Let's write our own function for returning bootstrap samples. Let's call our function `bootstrap()`. It should take two arguments, `data` and `n_boot`; the first is a 1-dimmensional array, and the second is an integer. Our function should return a 2-dimmensional NumPy array with `n` rows and `n_boot` columns, where `n` is the length of `data` array, and where each column is a bootstrap sample from the origin `data`. 
    
There are a few ways that we could do this, but we will almost certainly want to use the `np.random.choice()` function in NumPy to sample from a 1-D array.
    
**HINT**: Recall also that we can use the `np.zeros()` function to allocate an array of arbirary dimensions that's filled with zeros. This is frequently useful when we want to "pre-allocated" an array that we later fill.

In [ ]:
# Define our function here 


In [ ]:
# Testing our fuction
x = np.random.normal(0, 1, 1_000_000)

boot_samples = bootstrap(x, 100)

In [ ]:
# Should print `True` 3 times
print(boot_samples.shape == (1_000_000, 100))
print(-0.1 < np.mean(boot_samples) < 0.1)
print(0.9 < np.std(boot_samples) < 1.1)


# Random Forests
  * Ho (1995), Breiman (2001)
  * Extends idea of bagging
  * Build many trees, aggregate predictions
  * Only consider $m$ predictors at each node
      - These $m$ predictors are selected at random
  * Improve predictive performance

## Random Forests Pseudo-Code
![image](images/random_forests_pseudocode.png)

## Out-of-Bag (OOB) Samples
  * For each bootstrap iteration, we have some $(y_i, \bf{x_i})$  that weren't used in tree building
  * Use these $(y_i, \bf{x}_i)$ OOB samples to estimate test error
  * Also use these $(y_i, \bf{x_i})$ to generate measures of variable importance (more on this later)

## OOB and Test Error
![image](images/oob_error.png)

## Random Forests and Overfitting

  * It was once (mistakenly) believed that random forests would not over fit
  * You _can_ overfit by growing deep trees
  * Growing too many trees won't cause you to overfit (but it's wasteful of computing resources)
  * In general, people worry _much_ less about overfitting in random forests (relative to other approaches)

# Fitting Random Forest

Fitting random forests models using Python is fairly straightforward. There is an excellent implementation of random forests in the scikit-learn package.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from sklearn import metrics

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

## MNIST Data
* Handwritten digits
* Ten-class classification problem (i.e., 0-9 classification)

In [ ]:
# Load data from https://www.openml.org/d/554

X, y = fetch_openml('mnist_784', version = 1, return_X_y = True)

In [ ]:
# quick function to plot image from MNIST

def show_image(x):
    x_resize = np.array(x).reshape(28, 28)
    plt.imshow(x_resize, 
               cmap = "Blues")
    plt.show()

## Example Images

In [ ]:
n = 137                    # image number
show_image(X.iloc[n])      # plot image

In [ ]:
print(y[n])

## Split Training and Test Sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = 0.2, 
                                                    random_state = 0)

# Fitting our Random Forest Model

In [ ]:
# create model object

rfc = RandomForestClassifier(n_jobs = 2, 
                             n_estimators = 100,
                             oob_score = True)

In [ ]:
rfc.fit(X_train, y_train)     # fit our model (takes time)

In [ ]:
rfc.oob_score_               # Out-of-bag error estimate

In [ ]:
rfc.score(X_test, y_test)    # test error

## Contrast with Logistic Regression

In [ ]:
mod = LogisticRegression(max_iter = 10000, solver = "saga")       # create model object

mod.fit(X_train, y_train)               # fit model  

In [ ]:
y_pred = mod.predict(X_test)            # use fitted model to make predictions

metrics.accuracy_score(y_test, y_pred)

# Gradient Boosted Trees

* Boosted trees and random forests are both ensemble learning methods that combine multiple decision trees
* But boosted trees and random forests differ in several key aspects
  - _Tree-building strategy_: boosted trees are built sequentially
  - _Cominbation strategy_: boosted trees are combined using a weighted sum, where weights depend on performance of each tree in reducing the loss function. Random forests essentially do majority vote (classification) or average of predictions (regression)
  - _Overfitting_: Boosted trees are more prone to overfitting, especially when the number of trees is large or the learning rate is high. Random forests are less prone to overfitting, as the random feature selection at each split introduces diversity and reduces correlation among trees

## Gradient Boosted Tree Algorithms

* AdaBoost (Adaptive Boosting):
  - One of the earliest boosting algorithms
  - Uses a simple reweighting of the samples based on the errors made by the previous trees
  - Generally limited to binary classification problems

* Gradient Boosting Machines (GBMs):
  - Builds trees based on the gradient of the loss function, making it more general and applicable to various loss functions and problem types
  - Used for both classification and regression tasks
  - Introduces a learning rate parameter to control model complexity

* XGBoost (eXtreme Gradient Boosting):
  - An optimized and scalable version of GBMs
  - Offers additional regularization techniques to control overfitting, such as L1 and L2 regularization
  - Supports parallel and distributed computing, making it faster and more efficient
  - Can handle missing data and provides feature importance scores

### Fitting Boosted Tree Model in SciKit-Learn

In [ ]:
btc = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)

btc.fit(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)